In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.environ['KAGGLE_USERNAME'] = "haodong1997"
os.environ['KAGGLE_KEY'] = "f31c6aa3982c597ae202acc8a1427559"

!pip install --upgrade --force-reinstall --no-deps kaggle

# Download the dataset.zip into the current run time. (Check /content, it should be there)
!kaggle competitions download -c histopathologic-cancer-detection
!cp /content/histopathologic-cancer-detection.zip /content/
!unzip -qq /content/histopathologic-cancer-detection.zip -d /content/


Mounted at /content/drive
cp: cannot stat '/content/histopathologic-cancer-detection.zip': No such file or directory
unzip:  cannot find or open /content/histopathologic-cancer-detection.zip, /content/histopathologic-cancer-detection.zip.zip or /content/histopathologic-cancer-detection.zip.ZIP.


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
#from sklearn.utils import shuffle
import matplotlib.patches as patches
import cv2
import shutil

#Needed for correct pathing with different operating systems
from pathlib import Path

In [ ]:
#Setting up paths to images & labels of data (may need to change the paths)
data_path = Path('/content/drive/MyDrive/Comp_Medicine/project_data/train_labels.csv')
data = pd.read_csv(data_path)
train_path = Path('/content/drive/MyDrive/Comp_Medicine/project_data/train/')
test_path = Path('/content/drive/MyDrive/Comp_Medicine/project_data/test/')

#Make new folders. Remake them if they already exist (wipe data)

temp_dir = '/content/drive/MyDrive/Comp_Medicine/project_data_split/train/positive/'
if os.path.exists(temp_dir):
  shutil.rmtree(temp_dir)
else:
  os.makedirs(temp_dir)

temp_dir = '/content/drive/MyDrive/Comp_Medicine/project_data_split/train/negative/'
if os.path.exists(temp_dir):
  shutil.rmtree(temp_dir)
else:
  os.makedirs(temp_dir)

temp_dir = '/content/drive/MyDrive/Comp_Medicine/project_data_split//val/positive/'
if os.path.exists(temp_dir):
  shutil.rmtree(temp_dir)
else:
  os.makedirs(temp_dir)

temp_dir = '/content/drive/MyDrive/Comp_Medicine/project_data_split/val/negative/'
if os.path.exists(temp_dir):
  shutil.rmtree(temp_dir)
else:
  os.makedirs(temp_dir)

data_train_pos_path = Path('/content/drive/MyDrive/Comp_Medicine/project_data_split/train/positive/')
data_train_neg_path =Path('/content/drive/MyDrive/Comp_Medicine/project_data_split/train/negative/')
data_val_pos_path = Path('/content/drive/MyDrive/Comp_Medicine/project_data_split//val/positive/')
data_val_neg_path = Path('/content/drive/MyDrive/Comp_Medicine/project_data_split/val/negative/')

In [ ]:
#Showing the visualizations of an image.

fig, ax = plt.subplots(1,3, figsize=(20,8))
img=plt.imread('/content/drive/MyDrive/Comp_Medicine/project_data/train/0000da768d06b879e5754c43e2298ce48726f722.tif')

#Regular image using matplotlib.pyplot
ax[0].set_title('Normal')
ax[0].imshow(img)

#CV2 processes the image as a BGR image, so the colors are 'incorrect'.
img2 = cv2.imread('/content/drive/MyDrive/Comp_Medicine/project_data/train/0000da768d06b879e5754c43e2298ce48726f722.tif')
ax[1].set_title('CV2 Default (BGR)')
ax[1].imshow(img2)

#To fix, we can just split and reorder the image to standard RGB.
b,g,r = cv2.split(img2)
rgb_img2 = cv2.merge([r,g,b])
ax[2].set_title('CV2 RGB')
ax[2].imshow(rgb_img2)

In [ ]:
#In this challenge, we are tasked to examine the 32x32 px center of the full 96x96 px image.
fig, ax = plt.subplots(figsize = (20,8))
ax.imshow(img)
focus = patches.Rectangle((32,32),32,32, linewidth = 3, edgecolor = 'g',facecolor='none')
ax.add_patch(focus)

# Preprocessing

For now, we will just preprocess the training data by cropping the image for the 32x32 center. Of course, this means we are cutting out potential information, but we can go back to this later. We can also prevent overfitting the model by performing random minor augmentations/perturbations to the image for the model to train on.

**Make sure that you have the folders created**

## Image Cropping

In [ ]:
#Image Cropping Method

fig, ax = plt.subplots(1,2,figsize = (20,8))
img=plt.imread('/content/drive/MyDrive/Comp_Medicine/project_data/train/0000da768d06b879e5754c43e2298ce48726f722.tif')
ax[0].imshow(img)
focus = patches.Rectangle((32,32),32,32, linewidth = 3, edgecolor = 'g',facecolor='none')
ax[0].add_patch(focus)

img_cropped = img[31:64,31:64].copy()
ax[1].imshow(img_cropped)

In [ ]:
num_data = len(data)
#Shuffle Data
data2 = data.sample(frac = 1).reset_index(drop=True)
#Split data into training and validation sets
num_val_data = num_data//10
data_val = data2.iloc[:num_val_data,:]
data_train = data2.iloc[num_val_data:,:].reset_index(drop=True)

In [ ]:
#Preprocess and save images in validation set
for ind in data_val.index:
    img_id = str(data_val['id'][ind]) + '.tif'
    img_path = train_path / img_id
    
    #print(img_path)
    if (data_val['label'][ind] == 0):
        new_img_path = data_val_neg_path / img_id
    else:
        new_img_path = data_val_pos_path / img_id
    
    img=cv2.imread(str(img_path))
    img_cropped = img[31:64,31:64].copy()
    cv2.imwrite(str(new_img_path), img_cropped)

In [ ]:
#Preprocess and save images in training set
for ind in data_train.index:
    img_id = str(data_train['id'][ind]) + '.tif'
    img_path = train_path / img_id
    
    #print(img_path)
    if (data_train['label'][ind] == 0):
        new_img_path = data_train_neg_path / img_id
    else:
        new_img_path = data_train_pos_path / img_id
    
    img=cv2.imread(str(img_path))
    img_cropped = img[31:64,31:64].copy()
    cv2.imwrite(str(new_img_path), img_cropped)

In [ ]:
#To Clear all data from preprocessed folders, run this block.
#shutil.rmtree(data_val_neg_path)
#os.makedirs(data_val_neg_path)

#shutil.rmtree(data_val_pos_path)
#os.makedirs(data_val_pos_path)

#shutil.rmtree(data_train_neg_path)
#os.makedirs(data_train_neg_path)

#shutil.rmtree(data_train_pos_path)
#os.makedirs(data_train_pos_path)

## Elimination
Get rid of very dark and very bright images like the ones seen below. (Or perhaps mark them as negative...)

In [ ]:
# For example, these images are either too dark or too bright to be useful.

fig, ax = plt.subplots(1,2, figsize=(20,8))
preimg = cv2.imread('/content/drive/MyDrive/Comp_Medicine/project_data/train/9369c7278ec8bcc6c880d99194de09fc2bd4efbe.tif')
b,g,r = cv2.split(preimg)
img = cv2.merge([r,g,b])
ax[0].set_title('Dark')
ax[0].imshow(img)

preimg2 = cv2.imread('/content/drive/MyDrive/Comp_Medicine/project_data/train/f6f1d771d14f7129a6c3ac2c220d90992c30c10b.tif')
b,g,r = cv2.split(preimg2)
img2 = cv2.merge([r,g,b])
ax[1].set_title('Bright')
ax[1].imshow(img2)